In [21]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from ipywidgets import widgets

from sklearn import preprocessing 

import pandas as pd

import numpy as np
CountryContinent = pd.read_csv("CountryContinent.csv", sep = ';')
transfers = pd.read_csv("DATABASE.csv")

In [22]:
transfers['Position_B'] = transfers['Position'].replace(to_replace={'Right Winger' : 'Winger',
                                                                    'Central Midfield' : 'Midfield',
                                                                    'Attacking Midfield' : 'Midfield',
                                                                    'Centre-Back' : 'Defender',
                                                                    'Left Midfield' : 'Midfield',
                                                                    'Right-Back' : 'Side Back',
                                                                    'Centre-Forward' : 'Forward',
                                                                    'Left-Back' : 'Side Back',
                                                                    'Defensive Midfield' : 'Midfield',
                                                                    'Second Striker' : 'Forward',
                                                                    'Goalkeeper' : 'Goalkeeper',
                                                                    'Right Midfield' : 'Midfield',
                                                                    'Left Winger' : 'Winger',
                                                                    'Forward' : 'Forward',
                                                                    'Sweeper' : 'Defender',
                                                                    'Defender' : 'Defender',
                                                                    'Midfielder' : 'Midfield'})


Country = pd.DataFrame(transfers['Country_to'].append(transfers['Country_from']), columns = ["Country"]).drop_duplicates()

transfers = transfers.merge(CountryContinent, how = 'left', left_on = 'Country_to', right_on = 'Country')

transfers.drop(columns = ['Country'], inplace = True)
transfers.head()




,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee,Country_from,Country_to,Position_B,Continent
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000,Spain,Spain,Winger,Europe
1,Flávio Conceicao,Central Midfield,26,Dep. La Coruña,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,25000000,Spain,Spain,Midfield,Europe
2,James Rodríguez,Attacking Midfield,23,Monaco,Ligue 1,Real Madrid,LaLiga,2014-2015,60000000.0,75000000,France,Spain,Midfield,Europe
3,Raúl Albiol,Centre-Back,23,Valencia CF,LaLiga,Real Madrid,LaLiga,2009-2010,9000000.0,15000000,Spain,Spain,Defender,Europe
4,Santiago Solari,Left Midfield,23,Atlético Madrid,LaLiga2,Real Madrid,LaLiga,2000-2001,NaN,3500000,Spain,Spain,Midfield,Europe


In [23]:
transfTeamSales = transfers.groupby(['Team_from','Country_from']).agg(EarnedMoney=('Transfer_fee', 'sum'), NumSales=('Transfer_fee', 'size')).reset_index()
transfTeamPurchase = transfers.groupby(['Team_to','Country_to']).agg(SpentMoney=('Transfer_fee', 'sum'), NumSignings=('Transfer_fee', 'size')).reset_index()
TransfTeam = transfTeamSales.merge(transfTeamPurchase, left_on = ['Country_from', 'Team_from'], right_on = ['Country_to', 'Team_to'], how = 'outer' )
TransfTeam.loc[TransfTeam['Team_from'].isna(), 'Team_from'] = TransfTeam['Team_to']
TransfTeam.loc[TransfTeam['Country_from'].isna(), 'Country_from'] = TransfTeam['Country_to']
TransfTeam.drop(columns = ['Team_to', 'Country_to'], inplace = True)
TransfTeam.fillna(0, inplace = True)
TransfTeam.rename(columns = {'Team_from' : 'Team', 'Country_from' : 'Country'}, inplace = True)
print(TransfTeam.head())

             Team  Country  EarnedMoney  NumSales  SpentMoney  NumSignings
0      1. FC Köln  Germany   88600000.0      10.0  41500000.0          5.0
1  1.FC K'lautern  Germany   18100000.0       5.0  14300000.0          6.0
2  1.FC Nuremberg  Germany   47100000.0       9.0         0.0          0.0
3  1.FSV Mainz 05  Germany  140900000.0      16.0  35800000.0          6.0
4     1860 Munich  Germany   13650000.0       4.0   5575000.0          4.0


In [24]:
TransfTeam = TransfTeam.sort_values(['Country'])
Country_names = TransfTeam['Country'].unique()
Country_data = {Country:TransfTeam.query("Country == '%s'" %Country)
                              for Country in Country_names}
print(Country_data)

{'Argentina':                 Team    Country  EarnedMoney  NumSales  SpentMoney  \
105    CD Godoy Cruz  Argentina    4000000.0       1.0         0.0   
504            Tigre  Argentina    4000000.0       1.0         0.0   
552  Vélez Sarsfield  Argentina  102900000.0      18.0         0.0   
231         Gimnasia  Argentina    6000000.0       2.0         0.0   
75          Belgrano  Argentina    4500000.0       1.0         0.0   
257          Huracán  Argentina   22200000.0       3.0         0.0   
419      Racing Club  Argentina   92400000.0      17.0   5200000.0   
100      CA Banfield  Argentina   22650000.0       5.0   3500000.0   
137            Colón  Argentina   14000000.0       4.0   1800000.0   
103       CA Rosario  Argentina   36400000.0       8.0         0.0   
439      River Plate  Argentina  348230000.0      39.0  23600000.0   
40        Argentinos  Argentina   28450000.0       6.0         0.0   
122    Chacarita Jrs  Argentina    1300000.0       1.0         0.0   
405   

In [25]:
fig = go.Figure()

for Country_names, Country in Country_data.items():
    fig.add_trace(go.Scatter(type = 'scatter',
        x=Country['EarnedMoney'],
        y=Country['SpentMoney'],
        name=Country_names,
        text=Country['Team'],
        hovertemplate=
        '<b>%{text}</b>' +
        '<br><b>Spending</b>: €%{y}'+
        '<br><b>Earnings</b>: €%{x}'+
        '<br><b>Sales</b>: %{marker.size:}',
        mode = 'markers',
        marker = dict(size = Country['NumSignings'], 
                      sizeref = 2)
        ))

fig.update_layout(dict(title = 'Total Transfers Per Team 2000 - 2019',
                       xaxis = dict(title = 'Total Recived by Sales'),
                       yaxis = dict(title = 'Total Invested in Hiring')))
    
fig.show()

In [42]:
transfers = transfers.sort_values(['Age'])
Age_yrs = transfers['Age'].unique()
print(Age_yrs)
Age_data = {Age :transfers.query("Age == '%s'" %Age)
                              for Age in Age_yrs}
print(Age_data)

[ 0 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
{0:                    Name        Position  Age Team_from    League_from  \
4379  Marzouq Al-Otaibi  Centre-Forward    0    Shabab   Saudi Arabia   

      Team_to      League_to     Season  Market_value  Transfer_fee  \
4379  Ittihad   Saudi Arabia  2000-2001           NaN       2000000   

      Country_from    Country_to Position_B Continent  
4379  Saudi Arabia  Saudi Arabia    Forward    Arabic  , 15:              Name          Position  Age       Team_from League_from  \
4616  Fran Mérida  Central Midfield   15  Barcelona Yth.       Spain   

          Team_to League_to     Season  Market_value  Transfer_fee  \
4616  Arsenal U18   England  2005-2006           NaN       3200000   

     Country_from Country_to Position_B Continent  
4616        Spain    England   Midfield    Europe  , 16:                    Name            Position  Age        Team_from  \
242   Slobodan Rajkovic         Centre-Back   16      OFK

In [45]:
fig = go.Figure()

for Age_yrs, Age in Age_data.items():
    fig.add_trace(go.Scatter(type = 'scatter',
        x=Age['Transfer_fee'],
        y=Age['Position_B'],
        name=str(Age_yrs),
        text=Age['Name'],
        hovertemplate=
        '<b>%{text}</b>' +
        '<br><b>Spending</b>: €%{y}'+
        '<br><b>Earnings</b>: €%{x}'+
        '<br><b>Sales</b>: %{marker.size:}',
        mode = 'markers',
        #marker = dict(size = Position['NumSignings'], 
        #              sizeref = 2)
        ))

fig.update_layout(dict(title = 'Total Transfers Per Team 2000 - 2019',
                       xaxis = dict(title = 'Total Recived by Sales'),
                       yaxis = dict(title = 'Total Invested in Hiring')))
    
fig.show()

In [6]:
transfSeasonSales = transfers.groupby(['Country_to','Team_to','Season']).agg(MoneySpent=('Transfer_fee', 'sum'), NumHires=('Transfer_fee', 'size')).reset_index()
transfSeasonSales.rename(columns = {'Team_to' : 'Team', 'Country_to' : 'Country'}, inplace = True)
transfSeasonSales = transfSeasonSales.sort_values(['Season', 'MoneySpent'])
print(transfSeasonSales.head())

MinSeasonSale = transfers.sort_values(['Season','Transfer_fee'])
MinSeasonSale.drop_duplicates(subset=['Season'], keep='first', inplace=True)
MinSeasonSale['Age']=MinSeasonSale['Age'].apply(str)
MaxSeasonSale = transfers.sort_values(['Season','Transfer_fee'])
MaxSeasonSale.drop_duplicates(subset=['Season'], keep='last', inplace=True)
MaxSeasonSale['Age']=MaxSeasonSale['Age'].apply(str)
print(MinSeasonSale.head())

         Country           Team     Season  MoneySpent  NumHires
1288  Nederlands     AZ Alkmaar  2000-2001     1800000         1
943       Greece  PAOK Saloniki  2000-2001     1850000         1
815      Germany   E. Frankfurt  2000-2001     1880000         1
437      England     Preston NE  2000-2001     2000000         1
513      England        Watford  2000-2001     2000000         1
                  Name        Position Age       Team_from       League_from  \
1775          Yago Yao     Centre-Back  21  Sporting Gijón           LaLiga2   
2384       John Aloisi  Centre-Forward  25   Coventry City    First Division   
4431        Rémo Meyer     Centre-Back  21  Lausanne-Sport  Challenge League   
471           Bosingwa      Right-Back  20        Boavista          Liga NOS   
329   Juan Pablo Sorín       Left-Back  28        Cruzeiro            Brazil   

            Team_to     League_to     Season  Market_value  Transfer_fee  \
1775  Celta de Vigo        LaLiga  2000-2001         

In [7]:

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=transfSeasonSales['Season'] ,
                     y=transfSeasonSales['MoneySpent'],
                     name="Amount Spend by Team",
                     text=transfSeasonSales['Team']+ ' - ' +transfSeasonSales['Country'],
                     hovertemplate='<b>%{text}</b>' +
                                   '<br><b>Spending</b>: €%{y}',
                     marker_color='dimgray')
                     ,secondary_y=False,)
fig.add_trace(go.Scatter(x=MaxSeasonSale['Season'],
                         y=MaxSeasonSale['Transfer_fee'],
                        name="Highest Transfer",
                        text="Player: "+MaxSeasonSale['Name'] + ' ('+MaxSeasonSale['Age'] +"yrs) <br>"+
                             "From: "+MaxSeasonSale['Team_from']+" - "+MaxSeasonSale['Country_from']+" <br>"+
                             "To: "+MaxSeasonSale['Team_to']+" - "+MaxSeasonSale['Country_to'],
                        hovertemplate= '%{text}' +
                                       '<br><b>Transfer fee</b>: €%{y}',
                              marker_color='navy'
                        ),secondary_y=True)
fig.add_trace(go.Scatter(x=MinSeasonSale['Season'],
                         y=MinSeasonSale['Transfer_fee'],
                        name="Lowest Transfer",
                        text="Player: "+MinSeasonSale['Name'] + ' ('+MinSeasonSale['Age'] +"yrs) <br>"+
                             "From: "+MinSeasonSale['Team_from']+" - "+MinSeasonSale['Country_from']+" <br>"+
                             "To: "+MinSeasonSale['Team_to']+" - "+MinSeasonSale['Country_to'],
                        hovertemplate= '%{text}' +
                                       '<br><b>Transfer fee</b>: €%{y}',
                         marker_color='red'
                        ),secondary_y=True)

fig.update_layout(dict(title = 'Total Transfers by Team per Season',
                       xaxis = dict(title = 'Season'),
                       yaxis = dict(title = 'Transfer Fee Team'),
                       legend= dict(x=-.1, y=1.1),
                       legend_orientation="h"))

fig.update_yaxes(title_text="Highest\Lowest Transfer Fee ", secondary_y=True)

fig.show()

In [42]:
CountryRelations = transfers.groupby(['Country_to','Country_from']).agg(Transfer_fee=('Transfer_fee', 'sum'), Transactions = ('Transfer_fee', 'size')).reset_index()
#CountryRelations = CountryRelations[CountryRelations['Country_to'] != CountryRelations['Country_from']]


Country = pd.DataFrame(CountryRelations['Country_to'].append(CountryRelations['Country_from']), columns = ["Country"])
Country.drop_duplicates(inplace=True)

le = preprocessing.LabelEncoder()
Country["Country_ind"] = le.fit_transform(Country["Country"])
Country.sort_values(['Country_ind'], inplace = True)
CountryRelations = CountryRelations.merge(Country, how = 'left', left_on = 'Country_to', right_on = 'Country')
CountryRelations = CountryRelations.merge(Country, how = 'left', left_on = 'Country_from', right_on = 'Country')
CountryRelations.sort_values(['Transfer_fee'], inplace = True, ascending=False)
CountryRelations = CountryRelations.head(50)
print(CountryRelations.head())

    Country_to Country_from  Transfer_fee  Transactions Country_x  \
71     England      England    6505610000           682   England   
186      Italy        Italy    4505420000           465     Italy   
358      Spain        Spain    1869810000           179     Spain   
90     England        Spain    1721210000           110   England   
72     England       France    1689860000           142   England   

     Country_ind_x Country_y  Country_ind_y  
71              16   England             16  
186             25     Italy             25  
358             47     Spain             47  
90              16     Spain             47  
72              16    France             18  


In [45]:
fig = go.Figure(go.Sankey(
    valuesuffix = "€",
    # Define nodes
    node = dict(label = Country["Country"]
               ),
    # Add links
    link = dict(
          source =  CountryRelations["Country_ind_y"],
          target =  CountryRelations["Country_ind_x"],
          value =   CountryRelations["Transfer_fee"],
          hovertemplate = '%{x}' + '<br><b>Transfer fee</b>: €%{y}'
      )))



fig.show()


In [8]:
columnSet = ['Continent', 'Country_to', 'Team_to', 'Position_B', 'Name']
database = pd.DataFrame(columns= ['ids', 'labels', 'parents', 'transfers', 'transferfee'])
for i, columnName in enumerate(columnSet):
    database1 = pd.DataFrame(columns= ['ids', 'labels', 'parents', 'transfers', 'transferfee'])
    groupdf = transfers.groupby(columnSet[:i+1]).agg(transfers = ('Transfer_fee', 'size'), transferfee =  ('Transfer_fee', 'sum')).reset_index()
    if i == 0:
        database1['ids'] = groupdf[columnName]
        database1['labels'] = groupdf[columnName]
        database1['transfers'] = groupdf['transfers']
        database1['transferfee'] = groupdf['transferfee']
        database = database.append(database1)
    else:
        groupdf['combined1'] = groupdf[columnSet[:i+1]].apply(lambda row: ' - '.join(row.values.astype(str)), axis=1)
        groupdf['combined2'] = groupdf[columnSet[:i]].apply(lambda row: ' - '.join(row.values.astype(str)), axis=1)
        database1['ids'] = groupdf['combined1']
        database1['labels'] = groupdf[columnName]
        database1['parents'] = groupdf['combined2']
        database1['transfers'] = groupdf['transfers']
        database1['transferfee'] = groupdf['transferfee']
        database = database.append(database1)

In [13]:
fee_max = 30
fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=database.ids,
    labels=database.labels,
    parents=database.parents,
    values=database.transfers,
    branchvalues = 'total',
    marker=dict(
        cmin = transfers['Transfer_fee'].min(), 
        cmax = 1820000000,
        colors=database.transferfee,
        colorscale='OrRd'),
    hovertemplate='<b>%{label} </b> <br> Transfers: %{value}<br> Transfer fee: €%{color:,}',
    maxdepth=3
))
fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=0, l=0, r=0, b=0)
)

fig.show()